В качестве тегсета я беру частеречный инвентарь из ридинга, так как он был разработан как раз для соответсвующей задачи: сравнения нескольких морфанализаторов. существительные (S), прилагательные (A), глаголы, в том числе причастия и деепричастия (V), предлоги (PR), союзы (CONJ) и сборная категория, включающая прочие несклоняемые слова: наречия, вводные слова, частицы, междометия (ADV). Я также стала размечать местоимения, однако без уточнения их разряда (PRO).

Тексты были взяты с грамматической омонимией, как с одной из типовых ошибок, выявленных в соревновании "Диалог 2010"; аббревиатурой, которую система могла бы спутать с глаголами или словами других классов; дефисными написаниями, которые могли разбиваться/не разбиваться системой и дальше анализироваться, и с непроективными предложениями (стихи), так как я предположила, что нетипичная синтаксичсекая зависимость могла ввести морфанализатор в заблуждение.

In [1]:
!pip3 install nltk

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [71]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import csv

In [3]:
corpus = ['Перед нами была огромная пропасть.', 'Грибов там можно набрать пропасть.',
     'Тьфу ты, пропасть, как надоел!', 'В таком месте легко пропасть.',
     'Ты белых лебедей кормила, откинув тяжесть черных коc.',
     'Я рядом плыл, сошлись кормила.', 'Закатный луч был странно кос.',
     'Вдруг лебедей метнулась пара.', 'Не знаю, чья была вина.',
     'Закат замлел за дымкой пара, алея, как поток вина.', 
     'Мгновенья двигались и стали.', 'Лишь ты паришь, свой свет струя.',
     'Меж тем в реке – из сизой стали влачится за струей струя.',
     'Небесное светило было закрыто дождевыми тучами, а еще вчера оно так ярко и ласково нам светило.',
     'Если б мыло приходило по утрам ко мне в кровать и само меня бы мыло, хорошо бы это было.',
     'Мы ели ершей у ели.', 'Их еле-еле у ели доели.', 'Чай пили в саду', 'Пили скорей дрова.',
     'Белки прыгают по веткам, принесли орешки деткам!',
     'Взбей яичные белки, и печенье испеки.', 
     'В ведерко руку окуни, не бойся: это окуни.',
     'Из окна дуло.', 'Штирлиц закрыл окно.', 'Дуло исчезло.',
     'Она научила меня печь блины', 'Натопили печь и напекли картошки.',
     'Серьезную озабоченность продолжает вызывать транснациональная организованная преступность и связанная с ней контрабанда оружия, незаконная миграция и торговля людьми.',
     'Рост потребления наркотиков представляет реальную угрозу социально-экономическому развитию стран Содружества.',
     'К сожалению, приобретает международный характер и киберпреступность, которая посягает на национальную безопасность государств-участников СНГ.']

In [19]:
tokens = []
sw = [',', '.', ':', '!', '–']
for sent in corpus:
    tokens += [w for w in word_tokenize(sent) if w not in sw]  

In [58]:
len(tokens)

210

In [64]:
with open('prepre_tag.csv', 'w', encoding='utf-8') as w_file:
    file_writer = csv.writer(w_file, delimiter=',')
    for word in tokens:     
        file_writer.writerow([word, ''])

In [33]:
gold = []
gold_mystem = []
gold_spacy = []
with open('annotated_tags.csv', 'r', encoding='utf-8') as r_file:
    file_reader = csv.reader(r_file, delimiter=',')
    for row in file_reader:
        if row[0] == 'б':
            gold.append(row[1])
            gold_mystem.append(row[1])
        elif '-' in row[0]:
            gold.append(row[1])
            gold_mystem.append(row[1])
            for i in range(3):
                gold_spacy.append(row[1])
        else:
            gold.append(row[1])
            gold_mystem.append(row[1])
            gold_spacy.append(row[1])
            
                   
# mystem разбил дефисное слово "госудств-участников" по дефису (далее дефис не учитывался в разборе),
# а "еле-еле", "социально-экономическому" – нет,
# и определил у каждого элемента часть речи,
# поэтому кол-во слов в разметке стало на 1 больше, чем в gold.
# чтобы дальше провести сравнение, я решила задать еще один gold, который
# будет отличаться от исходного на одно существительное
# ("госудств-участников" – предпоследнее сущ перед "СНГ")
gold_mystem.append('S')

# spacy разбил каждое дефисное слово:
# "госудств-участников", "еле-еле", "социально-экономическому" с учетом дефиса,
# причем в только в случае "госудств-участников" и "социально-экономическому"
# все три составных элемента носят частеречный тег всего сочетания,
# то есть NOUN и ADJ соотвественно,
# но не стал учитывать "б" в "Если б мыло...",
# то есть кол-во слов в разборе стало на 5 больше, чем в gold

In [135]:
print(len(gold))
print(len(gold_mystem))
print(len(gold_spacy))

210
211
215


# Pymorphy, mystem, spacy

In [16]:
!pip3 install pymorphy2
!pip3 install pymystem3
!pip3 install setuptools wheel
!pip3 install spacy
!python3 -m spacy download ru_core_news_sm
import spacy
from pymystem3 import Mystem
m = Mystem()
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import re

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
     |████████████████████████████████| 16.4 MB 12.0 MB/s eta 0:00:01


You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [17]:
mystem_result = []
for sent in corpus:
    ana = m.analyze(sent)
    for word in ana:
        if 'analysis' in word:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            mystem_result.append(pos) # разметка Mystem

In [21]:
pymorphy_result = []
for token in tokens:
    ana2 = morph.parse(token)[0]
    find = re.findall('[A-Z]*', str(ana2.tag))
    pymorphy_result.append(find[0]) # разметка Pymorphy

In [22]:
nlp = spacy.load("ru_core_news_sm")

In [23]:
spacy_result = []
for sent in corpus:    
    doc = nlp(sent)
    exept = ['PUNCT', 'SPACE']
    for word in doc:
        if word.pos_ not in exept:
            spacy_result.append(word.pos_) # разметка Spacy

In [47]:
tags = {'S': ['NOUN', 'S', 'PROPN'],
        'PR': ['ADP', 'PREP', 'PR'],
        'ADV': ['PART', 'ADV', 'ADVB', 'PRCL', 'INTJ'],
        'V': ['VERB', 'PRED', 'AUX', 'V', 'PRTF', 'PRTS', 'INFN', 'GRND'],
        'A': ['ADJ', 'ADJF', 'A', 'ADJS'],
        'CONJ': ['CCONJ', 'SCONJ', 'CONJ'],
        'PRO': ['PRON', 'APRO', 'SPRO', 'ADVPRO', 'NPRO', 'DET']}

In [48]:
def convert(tag):
    for key in tags:
        if tag in tags[key]:
            tag = key
    return tag

In [49]:
conv_pymorphy_result = []
for tag in pymorphy_result:
    conv_pymorphy_result.append(convert(tag))

In [50]:
conv_mystem_result = []
for tag in mystem_result:
    conv_mystem_result.append(convert(tag))

In [51]:
conv_spacy_result = []
for tag in spacy_result:
    conv_spacy_result.append(convert(tag))

# Accuracy

In [28]:
!pip3 install sklearn
from sklearn.metrics import accuracy_score

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [53]:
print("RESULTS:")
print('Pymorphy')
print("Accuracy: %.4f" % accuracy_score(conv_pymorphy_result, gold)) 

RESULTS:
Pymorphy
Accuracy: 0.8381


In [57]:
print("RESULTS:")
print('Spacy')
print("Accuracy: %.4f" % accuracy_score(conv_spacy_result, gold_spacy)) 

RESULTS:
Spacy
Accuracy: 0.8791


In [54]:
print("RESULTS:")
print('Mystem')
print("Accuracy: %.4f" % accuracy_score(conv_mystem_result, gold_mystem)) 

RESULTS:
Mystem
Accuracy: 0.8957


# Chunker

Наречие *очень* встречается и с положительными, и с негативными отзывами, поэтому, чтобы различать случаи, предлагаю рассматреть сочетания, которые более тяготеют к отрицательному: очень + не + краткое A(довольна) или V. Так же вниманию стоит уделить такие сочетания как V + всего, так как ярко выраженного негатива нет, однако касательно робот-пылесосов работает/проработал всего.../заряд держит всего... – частый способ выразить недовольство товаром. Также такие сочетания как: не + V + раньше (жалею, что не появилось раньше/не знала  раньше, что такие бывают), так как по подсчету негативных слов засчет *не* тут больше, а смысл положительный.

In [133]:
def chunker(text):
    tokens =  [token.lower() for token in word_tokenize(text) if token.isalpha()]
    pos_tags = []
    for sent in sent_tokenize(text):
        ana = m.analyze(sent)
        for word in ana:
            if 'analysis' in word:
                gr = word['analysis'][0]['gr']
                pos = gr.split('=')[0].split(',')[0]
                pos_tags.append(pos)
    d = {}
    for i in range(len(tokens)):
        d[tokens[i]] = pos_tags[i]
    ngrams = []
    for token in tokens:
        if token == 'очень' and tokens[tokens.index('очень') + 1] == 'не' and d[tokens[tokens.index('очень') + 2]] == 'V':
            ngrams.append(token + ' ' + tokens[tokens.index('очень') + 1] + ' ' + tokens[tokens.index('очень') + 2])
        elif token == 'очень' and tokens[tokens.index('очень') + 1] == 'не' and d[tokens[tokens.index('очень') + 2]] == 'A':
            ngrams.append(token + ' ' + tokens[tokens.index('очень') + 1] + ' ' + tokens[tokens.index('очень') + 2])
        elif token == 'всего' and d[tokens[tokens.index('всего') - 1]] == 'V':
            ngrams.append(tokens[tokens.index('всего') - 1] + ' ' + token)
        elif token == 'раньше' and d[tokens[tokens.index('раньше') - 1]] == 'V' and tokens[tokens.index('раньше') - 2] == 'не':
            ngrams.append(tokens[tokens.index('раньше') - 2] + ' ' + tokens[tokens.index('раньше') - 1] + ' ' + token)
    return ngrams